### LOADING DATA

In [3]:
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader(r"C:\Users\LENOVO\Desktop\ML_RAG\donnee_use_case_structuré.pdf")
data = loader.load()
print(len(data))
print(type(data))


89
<class 'list'>


### SPLIT THE DATA

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(data)

print((chunks[0].page_content))
print(type(chunks))

Données d'utilisation des clients
Univers : Lignes
Description : Identifiant de la ligne (numéro de téléphone)
Utilisé pour effectuer la segmentation : Oui
  Predire le Churn : Oui
Univers : Lignes
Description : Type de ligne (prépayé ou postpayé)
Utilisé pour effectuer la segmentation : Oui
  Predire le Churn : Non
Univers : Lignes
Description : Ancienneté de la ligne en jours (date courante - date d'activation de la ligne)
Utilisé pour effectuer la segmentation : Oui
  Predire le Churn : Oui
Univers : Lignes
Description : Date d'activation de la ligne
Utilisé pour effectuer la segmentation : Oui
  Predire le Churn : Oui
Univers : Lignes
Description : Date de résiliation de la ligne
Utilisé pour effectuer la segmentation : Non
  Predire le Churn : Oui
<class 'list'>


### Vector store

In [5]:
from langchain_nomic import NomicEmbeddings
from langchain_community.vectorstores import FAISS
import os

os.environ["NOMIC_API_KEY"] = "nk-uC7cI5olWsQ8GFL5d_y54N5zpNlx9vH86SgOieRzBWA"
embeddings = NomicEmbeddings(model="nomic-embed-text-v1.5")


vector_store = FAISS.from_documents(documents=chunks, embedding=embeddings)
print(type(vector_store))

<class 'langchain_community.vectorstores.faiss.FAISS'>


### Retriever 

In [28]:
retriever = vector_store.as_retriever()


### QUERY

In [27]:
# -----------QUERY--------------
query = "Quel sont les variables pour predire le churn "

# retrieved_docs = retriever.invoke(query)
retrieved_docs = vector_store.similarity_search(query, k=5)

context = "\n".join([doc.page_content for doc in retrieved_docs])
prompt = f"Contexte:\n{context}\n\nQuestion: {query}\rReponse:"
# ----------------------------

In [32]:
from langchain_core.prompts import PromptTemplate

template = """Utilisez le contexte suivant comme information utile, mais n'hésitez pas à utiliser vos propres connaissances pour améliorer la réponse.
Si le contexte est insuffisant ou sans rapport, fiez-vous à votre compréhension pour fournir la meilleure réponse possible.

{context}

Question: {question}

Reponse:"""

custom_rag_prompt = PromptTemplate.from_template(template)
print(custom_rag_prompt.format(context=context, question=query))



Utilisez le contexte suivant comme information utile, mais n'hésitez pas à utiliser vos propres connaissances pour améliorer la réponse.
Si le contexte est insuffisant ou sans rapport, fiez-vous à votre compréhension pour fournir la meilleure réponse possible.

Predire le Churn : Oui
Univers : Montant
Description : Revenu total du mois M-3
Utilisé pour effectuer la segmentation : Non
  Predire le Churn : Oui
Univers : Montant
Description : Revenu total du mois M-4
Utilisé pour effectuer la segmentation : Non
  Predire le Churn : Oui
Univers : Montant
Description : Revenu total du mois M-5
Utilisé pour effectuer la segmentation : Non
  Predire le Churn : Oui
Univers : Montant
Description : Revenu total du mois M-6
Utilisé pour effectuer la segmentation : Non
  Predire le Churn : Oui
Univers : Montant
Description : Revenu généré par les appels internationaux de type voix lors du mois M-1
Utilisé pour effectuer la segmentation : Non
  Predire le Churn : Oui
Predire le Churn : Non
Predire 

### LLAMA

In [33]:
from langchain_ollama.llms import OllamaLLM

llm=OllamaLLM(model='llama3.2:latest')
response = llm.invoke(custom_rag_prompt.format(context=context, question=query))
print("Answer:", response)

Answer: Selon les informations fournies, les variables suivantes peuvent être utilisées pour prédire le churn :

1. Revenu total du mois M-3
2. Revenu total du mois M-4
3. Revenu total du mois M-5
4. Revenu total du mois M-6
5. Revenu généré par les appels internationaux de type voix lors du mois M-1
6. Nombre total de points de fidélité utilisés lors des 6 derniers mois
7. Mode de paiement actuel
8. Part de la durée des appels vocaux sortants facturés parmi la durée du facturé total durant les 6 derniers mois
9. Nombre moyen de recharges durant les 6 derniers mois
10. Montant moyen des recharges durant les 6 derniers mois

Cependant, il est important de noter que certaines variables ne sont pas utilisées pour effectuer la segmentation, mais peuvent néanmoins avoir un impact sur le churn. C'est le cas :

* Revenu total de SMS nationaux envoyés vers mobiles IAM
* Revenu total de SMS nationaux envoyés vers n° spéciaux
* Revenu total de SMS nationaux envoyés vers autres n° (roaming, fax)


### OPENAI

In [ ]:
# ---------------- WE HAVE TO PAY TO USE OPENAI --------------------

# os.environ['OPENAI_API_KEY']='Your_openai_ApiKey'
# from langchain_openai import OpenAI
# llm3 = OpenAI()